In [5]:
import os, sys
import pandas as pd
import numpy as np
import backtrader as bt
# import setup_psql_environment
# from models import Security, SecurityPrice
from scipy.stats import linregress
from collections import defaultdict
from tabulate import tabulate
import PyQt5
import matplotlib
import datetime
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import backtrader.plot
from matplotlib.pyplot import figure

etf_tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']

In [6]:
def momentum_func(self, price_array):
    r = np.log(price_array)
    slope, _, rvalue, _, _ = linregress(np.arange(len(r)), r)
    annualized = (1 + slope) ** 252
    return (annualized * (rvalue ** 2))


class Momentum(bt.ind.OperationN):
    lines = ('trend',)
    params = dict(period=90)
    func = momentum_func

In [7]:
data = bt.feeds.YahooFinanceData(dataname='SPY',
                                  fromdate=datetime.datetime(2017, 1, 1),
                                  todate=datetime.datetime(2017, 12, 31))

In [11]:
class Strategy(bt.Strategy):
    params = dict(
        momentum=Momentum,
        momentum_period=180,
        num_positions=2,
        when=bt.timer.SESSION_START,
        timer=True,
        monthdays=[1],
        monthcarry=True,
        printlog=True
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.i = 0
        self.securities = self.datas[1:]
        self.inds = {}

        self.add_timer(
            when=self.p.when,
            monthdays=self.p.monthdays,
            monthcarry=self.p.monthcarry
        )

        for security in self.securities:
            self.inds[security] = self.p.momentum(security,
                                                  period=self.p.momentum_period)

    def notify_timer(self, timer, when, *args, **kwargs):
        if self._getminperstatus() < 0:
            self.rebalance()

    def rebalance(self):
        rankings = list(self.securities)
        rankings.sort(key=lambda s: self.inds[s][0], reverse=True)
        pos_size = 1 / self.p.num_positions

        # Sell stocks no longer meeting ranking filter.
        for i, d in enumerate(rankings):
            if self.getposition(d).size:
                if i > self.p.num_positions:
                    self.close(d)

        # Buy and rebalance stocks with remaining cash
        for i, d in enumerate(rankings[:self.p.num_positions]):
            self.order_target_percent(d, target=pos_size)

    def next(self):
        self.notify_timer(self, self.p.timer, self.p.when)

    def stop(self):
        self.log('| %2d | %2d |  %.2f |' %
                 (self.p.momentum_period,
                  self.p.num_positions,
                  self.broker.getvalue()),
                 doprint=True)

In [12]:
cerebro = bt.Cerebro()
cerebro.addstrategy(Strategy)
cerebro.broker.setcash(1337.0)
cerebro.broker.setcommission(commission=0.001)

In [13]:
cerebro.adddata(data)

In [14]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1337.00


In [15]:
cerebro.run()

2017-12-29, | 180 |  2 |  1337.00 |


In [16]:
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

Ending Portfolio Value: 1337.00


In [17]:
cerebro.plot(width=22,height=12)

C:\anaconda\envs\quantum\lib\site-packages\backtrader\plot\plot.py:127: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('nbagg')


[[<Figure size 640x480 with 4 Axes>]]